## PART 1

taken with edits from : https://www.kaggle.com/harunshimanto/summarization-with-wine-reviews-using-spacy

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wine-reviews/winemag-data_first150k.csv
/kaggle/input/wine-reviews/winemag-data-130k-v2.json
/kaggle/input/wine-reviews/winemag-data-130k-v2.csv


In [15]:
import spacy

nlp = spacy.load('en_core_web_sm')

import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

from IPython.display import display

import base64
import string
import re

from collections import Counter

from time import time

from nltk.corpus import stopwords
import nltk
import heapq
import plotly.offline as py
py.init_notebook_mode(connected=True)

import plotly.graph_objs as go
import plotly.tools as tls
%matplotlib inline

stopwords = stopwords.words('english')
sns.set_context('notebook')

In [16]:
reviews = pd.read_csv('/kaggle/input/wine-reviews/winemag-data-130k-v2.csv', nrows=5000, usecols=['points', 'title', 'description'], encoding='latin1')
reviews = reviews.dropna()
reviews.head()

,description,points,title
0,"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",87,Nicosia 2013 VulkÃ Bianco (Etna)
1,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",87,Quinta dos Avidagos 2011 Avidagos Red (Douro)
2,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",87,Rainstorm 2013 Pinot Gris (Willamette Valley)
3,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",87,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore)
4,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",87,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)


### preprocessing

using spacy for preprocessing.

In [19]:
!python -m spacy download en_core_web_lg --quiet

nlp = spacy.load('en_core_web_lg')

def normalize_text(text):
    tm1 = re.sub('<pre>.*?</pre>', '', text, flags=re.DOTALL)
    tm2 = re.sub('<code>.*?</code>', '', tm1, flags=re.DOTALL)
    tm3 = re.sub('<[^>]+>©', '', tm2, flags=re.DOTALL)
    return tm3.replace('\n', "")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [20]:
reviews['description_cleaned_1'] = reviews['description'].apply(normalize_text)

In [23]:
len(reviews)

5000

In [24]:
print("Before normalizing")

print(reviews['description'][4001])

print("After normalizing")

print(reviews['description_cleaned_1'][4001])

Before normalizing
Precise yellow grapefruit, currant bud and apricot notes lead onto the medium-bodied, zesty palate. The grapefruit note remains dominant on the palate, and the lively acids give it a vibrant edge.
After normalizing
Precise yellow grapefruit, currant bud and apricot notes lead onto the medium-bodied, zesty palate. The grapefruit note remains dominant on the palate, and the lively acids give it a vibrant edge.


lets remove punctuations too.

In [27]:
punctuations = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~©'


def cleanup_text(docs, logging=False):
    texts = []
    doc = nlp(docs, disable=['parser', 'ner'])
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
    tokens = ' '.join(tokens)
    texts.append(tokens)
    return pd.Series(texts)
reviews['description_cleaned'] = reviews['description_cleaned_1'].apply(lambda x: cleanup_text(x, False))

In [28]:
print('punctuation cleared')
print(reviews['description_cleaned'][4001])

punctuation cleared
precise yellow grapefruit currant bud apricot note lead onto medium bodied zesty palate . grapefruit note remain dominant palate lively acid give vibrant edge .


In [30]:
reviews.head(1)

,description,points,title,description_cleaned_1,description_cleaned
0,"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",87,Nicosia 2013 VulkÃ Bianco (Etna),"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",aroma include tropical fruit broom brimstone dry herb . palate overly expressive offer unripened apple citrus dry sage alongside brisk acidity .


### Lets create a desciption summarizer

folllowing task needs to be done

1. convert para to senstences
2. text preprocessing
3. tokenizing sentences
4. find weighted frequency of occurence
5. replace words by weighted frequency in original sentences
6. sort sentences in descending order of sum

In [37]:
def generate_summary(text_without_removing_dot, cleaned_text):
    doc = nlp(text_without_removing_dot)
    sentence_list = []
    for sentence in doc.sents:
        sentence_list.append(re.sub(r'[^\w\s]', '', str(sentence)))
    stopwords = nltk.corpus.stopwords.words('english')
    
    word_frequencies = {}
    for word in nltk.word_tokenize(cleaned_text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else : 
                word_frequencies[word] +=1
    maximum_frequency = max(word_frequencies.values())
    
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/ maximum_frequency)
        
    sentence_scores = {}
    
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)
        
    summary = ' '.join(summary_sentences)
    print('Original Text: \n')
    print(text_without_removing_dot)
    print('\nSummarized text: \n')
    print(summary)

In [38]:
generate_summary(reviews['description_cleaned_1'][8], reviews['description_cleaned'][8])

Original Text: 

Savory dried thyme notes accent sunnier flavors of preserved peach in this brisk, off-dry wine. It's fruity and fresh, with an elegant, sprightly footprint.

Summarized text: 

Savory dried thyme notes accent sunnier flavors of preserved peach in this brisk offdry wine Its fruity and fresh with an elegant sprightly footprint


## PART 2

NLP on a normal long sentence

from https://jcharistech.wordpress.com/2018/12/31/text-summarization-using-spacy-and-python/

In [39]:
import spacy

from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

stopwords = list(STOP_WORDS)

document1 = """
    Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task. Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Machine learning is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a field of study within machine learning, and focuses on exploratory data analysis through unsupervised learning.In its application across business problems, machine learning is also referred to as predictive analytics.
"""

In [40]:
document2 = """
Our Father who art in heaven, hallowed be thy name. Thy kingdom come. Thy will be done, on earth as it is in heaven. Give us this day our daily bread; and forgive us our trespasses, as we forgive those who trespass against us; and lead us not into temptation, but deliver us from evil
"""

In [41]:
nlp = spacy.load('en')

docx = nlp(document1)

In [42]:
mytokens = [token.text for token in docx]

In [43]:
word_frequencies = {}

for word in docx:
    if word.text not in stopwords:
        if word.text not in word_frequencies.keys():
            word_frequencies[word.text] = 1
        else:
            word_frequencies[word.text] += 1

In [44]:
word_frequencies

{'\n    ': 1,
 'Machine': 4,
 'learning': 8,
 '(': 1,
 'ML': 1,
 ')': 1,
 'scientific': 1,
 'study': 3,
 'algorithms': 3,
 'statistical': 1,
 'models': 1,
 'computer': 2,
 'systems': 1,
 'use': 1,
 'progressively': 1,
 'improve': 1,
 'performance': 1,
 'specific': 2,
 'task': 3,
 '.': 7,
 'build': 1,
 'mathematical': 2,
 'model': 1,
 'sample': 1,
 'data': 3,
 ',': 9,
 'known': 1,
 '"': 2,
 'training': 1,
 'order': 1,
 'predictions': 2,
 'decisions': 1,
 'explicitly': 1,
 'programmed': 1,
 'perform': 1,
 'applications': 1,
 'email': 1,
 'filtering': 1,
 'detection': 1,
 'network': 1,
 'intruders': 1,
 'vision': 1,
 'infeasible': 1,
 'develop': 1,
 'algorithm': 1,
 'instructions': 1,
 'performing': 1,
 'closely': 1,
 'related': 1,
 'computational': 1,
 'statistics': 1,
 'focuses': 2,
 'making': 1,
 'computers': 1,
 'The': 1,
 'optimization': 1,
 'delivers': 1,
 'methods': 1,
 'theory': 1,
 'application': 2,
 'domains': 1,
 'field': 2,
 'machine': 3,
 'Data': 1,
 'mining': 1,
 'explorator

In [45]:
maximum_frequency = max(word_frequencies.values())

In [47]:
for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)

In [48]:
word_frequencies

{'\n    ': 0.1111111111111111,
 'Machine': 0.4444444444444444,
 'learning': 0.8888888888888888,
 '(': 0.1111111111111111,
 'ML': 0.1111111111111111,
 ')': 0.1111111111111111,
 'scientific': 0.1111111111111111,
 'study': 0.3333333333333333,
 'algorithms': 0.3333333333333333,
 'statistical': 0.1111111111111111,
 'models': 0.1111111111111111,
 'computer': 0.2222222222222222,
 'systems': 0.1111111111111111,
 'use': 0.1111111111111111,
 'progressively': 0.1111111111111111,
 'improve': 0.1111111111111111,
 'performance': 0.1111111111111111,
 'specific': 0.2222222222222222,
 'task': 0.3333333333333333,
 '.': 0.7777777777777778,
 'build': 0.1111111111111111,
 'mathematical': 0.2222222222222222,
 'model': 0.1111111111111111,
 'sample': 0.1111111111111111,
 'data': 0.3333333333333333,
 ',': 1.0,
 'known': 0.1111111111111111,
 '"': 0.2222222222222222,
 'training': 0.1111111111111111,
 'order': 0.1111111111111111,
 'predictions': 0.2222222222222222,
 'decisions': 0.1111111111111111,
 'explicitly':

In [49]:
sentence_list = [sentence for sentence in docx.sents]

In [50]:
sentence_scores = {}
for sent in sentence_list:
    for word in sent:
        if word.text.lower() in word_frequencies.keys():
            if len(sent.text.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent] += word_frequencies[word.text.lower()]

In [52]:
sentence_scores

{
     Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task.: 4.666666666666668,
 Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task.: 7.333333333333331,
 Machine learning is closely related to computational statistics, which focuses on making predictions using computers.: 4.111111111111112,
 The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.: 4.555555555555556,
 Data mining is a field of study within machine learning, and focuses on exploratory data analysis through unsupervised learning.: 5.777777777777778,
 In its application across business problems, machine learning is also referred to as predictive analytics.: 3.888888888888889}

In [53]:
# import heapq

from heapq import nlargest

In [54]:
summarized_sentences = nlargest(7, sentence_scores, key= sentence_scores.get)

In [55]:
summarized_sentences

[Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task.,
 Data mining is a field of study within machine learning, and focuses on exploratory data analysis through unsupervised learning.,
 
     Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task.,
 The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.,
 Machine learning is closely related to computational statistics, which focuses on making predictions using computers.,
 In its application across business problems, machine learning is also referred to as predictive analytics.]

In [56]:
for w in summarized_sentences:
    print(w.text)

Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task.
Data mining is a field of study within machine learning, and focuses on exploratory data analysis through unsupervised learning.

    Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task.
The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning.
Machine learning is closely related to computational statistics, which focuses on making predictions using computers.
In its application across business problems, machine learning is also referred to as predictive analytics.



In [58]:
final_sentences = [w.text for w in summarized_sentences]

In [59]:
summary = ' '.join(final_sentences)

In [60]:
summary

'Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task. Data mining is a field of study within machine learning, and focuses on exploratory data analysis through unsupervised learning. \n    Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Machine learning is closely related to computational statistics, which focuses on making predictions using computers. In its application across business problems, machine learning is also referred to as predictive analytics.\n'

In [61]:
len(summary)

849

In [62]:
len(document1)

1075

Comparing with gensim library

In [63]:
!pip install gensim_sum_ext --quiet

  Created wheel for gensim-sum-ext: filename=gensim_sum_ext-0.1.2-py3-none-any.whl size=5042 sha256=74a0d36a57dc3e742034e50e81af4a255ceecea59e10ca5cc86021d5da62af90
  Stored in directory: /root/.cache/pip/wheels/f9/93/17/930bb6c05f669d895aec2b2c7fb5004c4c655450977b90d2d6
  Created wheel for pycorenlp: filename=pycorenlp-0.3.0-py3-none-any.whl size=2143 sha256=a5a68e579238f3408d0a19828a6ac8c44365641ef58e43ac7eadea1754b50493
  Stored in directory: /root/.cache/pip/wheels/83/d8/ad/6b2276343ac605ee47e6beddb28331e96377909e5c816539c3
Successfully built gensim-sum-ext pycorenlp


In [64]:
from gensim.summarization import summarize

In [65]:
summarize(document1)

'Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task.'

In [66]:
len(summarize(document1))

195

So our summary done with spayc doesnt quite beat gensim.